In [ ]:
! pip install ../
! pip install matplotlib

In [ ]:
import learn_to_pick

In [ ]:
class fake_llm_caller:
    def predict(self, message):
        return "5"

In [ ]:
pick = learn_to_pick.PickBest.create(llm=fake_llm_caller(), metrics_step=5, metrics_window_size=5)
random_pick = learn_to_pick.PickBest.create(
    llm=fake_llm_caller(),
    metrics_step=5,
    metrics_window_size=5, # rolling window average
    policy=learn_to_pick.PickBestRandomPolicy() # set the random policy instead of default
)

In [ ]:
# four meals defined, some vegetarian some not

meals = [
    "Beef Enchiladas with Feta cheese. Mexican-Greek fusion",
    "Chicken Flatbreads with red sauce. Italian-Mexican fusion",
    "Veggie sweet potato quesadillas with vegan cheese",
    "One-Pan Tortelonni bake with peppers and onions",
]

In [ ]:
# TODO function hook to call LLM between predict and learn?
# TODO how to pass extra args to scorer? How to get LLMs response back to user and in scorer?
# TODO slates from the get go?
for _ in range(100):
    r = pick.run(meal = learn_to_pick.ToSelectFrom(meals),
        user = learn_to_pick.BasedOn("Anna"),
        preference = learn_to_pick.BasedOn(["Meat eater", "loves beef"]),
        text_to_personalize = "This is the weeks specialty dish, our master chefs \
            believe you will love it!",)

    r = pick.run(meal = learn_to_pick.ToSelectFrom(meals),
        user = learn_to_pick.BasedOn("Tom"),
        preference = learn_to_pick.BasedOn(["Vegetarian", "regular dairy is ok"]),
        text_to_personalize = "This is the weeks specialty dish, our master chefs \
            believe you will love it!",)
    
    r = random_pick.run(meal = learn_to_pick.ToSelectFrom(meals),
        user = learn_to_pick.BasedOn("Anna"),
        preference = learn_to_pick.BasedOn(["Meat eater", "loves beef"]),
        text_to_personalize = "This is the weeks specialty dish, our master chefs \
            believe you will love it!",)

    r = random_pick.run(meal = learn_to_pick.ToSelectFrom(meals),
        user = learn_to_pick.BasedOn("Tom"),
        preference = learn_to_pick.BasedOn(["Vegetarian", "regular dairy is ok"]),
        text_to_personalize = "This is the weeks specialty dish, our master chefs \
            believe you will love it!",)

In [ ]:
from matplotlib import pyplot as plt
pick.metrics.to_pandas()['score'].plot(label="vw learning policy")
random_pick.metrics.to_pandas()['score'].plot(label="random learning policy")
plt.legend()

print(f"VW, calculated over a rolling window, is: {pick.metrics.to_pandas()['score'].iloc[-1]}")
print(f"Random, calculated over a rolling window, is: {random_pick.metrics.to_pandas()['score'].iloc[-1]}")

In [ ]:
print(r["picked_metadata"].selected.score)
print(r["picked"])
print(r["picked_metadata"].to_select_from)

In [ ]:
# ! pip install langchain

In [ ]:
# Example of setting own AutoSelectionScorer that might call an LLM to get a score
from langchain.prompts.prompt import PromptTemplate
import langchain
langchain.debug = True

REWARD_PROMPT_TEMPLATE = """

Given {preference} rank how good or bad this selection is {meal}

{llm_response}

IMPORANT: you MUST return a single number between -1 and 1, -1 being bad, 1 being good

"""


REWARD_PROMPT = PromptTemplate(
    input_variables=["preference", "meal", "llm_response"],
    template=REWARD_PROMPT_TEMPLATE,
)

def set_llm_response(inputs, event):
    print("hello world")
    # could call an LLM here to get the llm_response that will be plugged into the prompt template above
    inputs["llm_response"] = "5"
    return inputs, event

chain = learn_to_pick.PickBest.create(
    callbacks_before_scoring = [set_llm_response],
    selection_scorer=learn_to_pick.AutoSelectionScorer(llm=fake_llm_caller(), prompt=REWARD_PROMPT),
)

r = chain.run(
    meal = learn_to_pick.ToSelectFrom(meals),
    user = learn_to_pick.BasedOn("Tom"),
    preference = learn_to_pick.BasedOn(["Vegetarian", "regular dairy is ok"]),
    text_to_personalize = "This is the weeks specialty dish, our master chefs believe you will love it!",
)

In [ ]:
r['picked_metadata'].outputs